In [ ]:
import pickle
import numpy as np

import multiprocessing
import os
import time
import numpy as np
from scipy.spatial import cKDTree
import open3d as o3d
import util
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib import cm
import random
from BendLength import BendLengthCalculator
import pandas as pd
from IPython.display import display
import scipy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
point_cloud_location = "/home/chris/Code/PointClouds/sampled_model_dense.ply"
pcd = o3d.io.read_point_cloud(point_cloud_location)

# Preprocess the point cloud
pcd = util.preProcessCloud(pcd)
pcd_points = np.asarray(pcd.points)

In [3]:
with open("bend_visualization_data_all.pkl", "rb") as f:
    data = pickle.load(f)

bend_data = data["bend_data"] # ← This is what you were missing
all_normals = data["all_normals"]
segment_indices = data["segment_indices"]

In [ ]:
class ArcLengthCalculator:
    def __init__(self, pcd, bend_id, intersection_line, bend_edges, bend_cluster_points, cluster_derivatives, all_normals, segment_indices, intersection_step_size=0.5, derivative_step_size=0.01, radius=0.5):
        self.pcd = pcd
        self.points = np.asarray(pcd.points)

        if len(pcd.points) != len(all_normals):
            raise ValueError(f"Mismatch: {len(pcd.points)} points in PCD, but {len(all_normals)} normals loaded. "
                     "Make sure the point cloud is preprocessed the same way.")
        else:
            print(f"Loaded {len(all_normals)} normals for {len(pcd.points)} points in PCD. Proceeding.")
        
        self.bend_id = bend_id
        self.intersection_line = intersection_line
        self.bend_cluster_points = bend_cluster_points
        self.bend_cluster_derivatives = cluster_derivatives
        self.bend_cluster_kdtree = cKDTree(self.bend_cluster_points)
        self.pcd_kdtree = cKDTree(self.points)
        self.start_point, self.end_point = map(np.array, bend_edges)
        self.intersection_step_size = intersection_step_size
        self.derivative_step_size = derivative_step_size
        self.all_normals = all_normals
        self.segment_indices = segment_indices
        self.radius = 0.5
        self.intersection_it = 0
        self.derivative_it = 0
        self.is_moving_forward = True

        # self.pcd.paint_uniform_color([0.6, 0.6, 0.6])
        # self.vis = o3d.visualization.Visualizer()
        # self.vis.create_window("ArcLengthCalculator")
        # self.vis.add_geometry(self.pcd)

        self.plane_dominant_bins = {}
        for plane_id in [0, bend_id]:
            self.plane_dominant_bins[plane_id] = self.get_dominant_bin_ranges(
                self.all_normals[self.segment_indices[plane_id]],
                threshold=0.01
            )

    def plot_histogram_with_dominant_bins(self, angles, bin_edges, dominant_ranges, title, color):
        import matplotlib.pyplot as plt
        counts, edges, bars = plt.hist(angles, bins=bin_edges, color=color, edgecolor='black')
        # for i, bar in enumerate(bars):
        #     bin_start = edges[i]
        #     bin_end = edges[i+1]
        #     for dom_start, dom_end in dominant_ranges:
        #         if np.isclose(bin_start, dom_start, atol=1e-6) and np.isclose(bin_end, dom_end, atol=1e-6):
        #             bar.set_color('green')
        plt.title(title)
        plt.tight_layout()
        plt.show()

    def get_dominant_bin_ranges(self, normals, tolerance=0.01):
        dominant_bins = {}

        # Azimuth (angle in XY-plane)
        azimuth = np.degrees(np.arctan2(normals[:, 1], normals[:, 0]))
        azimuth = (azimuth + 360) % 360
        az_mode = float(scipy.stats.mode(np.round(azimuth, 2), keepdims=False).mode)
        az_range = [(az_mode - tolerance + 360) % 360, (az_mode + tolerance + 360) % 360]
        dominant_bins["azimuth"] = [tuple(az_range)]

        # Zenith (angle from Z axis)
        zenith = np.degrees(np.arccos(np.clip(normals[:, 2], -1, 1)))
        zen_mode = float(scipy.stats.mode(np.round(zenith, 2), keepdims=False).mode)
        zen_range = [zen_mode - tolerance, zen_mode + tolerance]
        dominant_bins["zenith"] = [tuple(zen_range)]

        print(f"Plane dominant azimuth mode: {az_mode:.2f}° ± {tolerance}°")
        print(f"Plane dominant zenith mode:  {zen_mode:.2f}° ± {tolerance}°")
        
        return dominant_bins

    def is_normal_in_any_bin(self, normal, bin_ranges, mode):
        if mode == "azimuth":
            angle = (np.degrees(np.arctan2(normal[1], normal[0])) + 360) % 360
        elif mode == "zenith":
            angle = np.degrees(np.arccos(np.clip(normal[2], -1, 1)))
        else:
            raise ValueError("Invalid mode: choose 'azimuth' or 'zenith'")
        
        #print(f"Angle: {angle} degrees")
        #print(f"Bin ranges: {bin_ranges}")
        for start, end in bin_ranges:
            start = (start + 360) % 360
            end = (end + 360) % 360
            if self.angle_in_range(angle, start, end):
                return True
        return False
    
    def angle_in_range(self, angle, start, end):
        """Checks if angle (in degrees) is inside [start, end], handling wrap-around."""
        angle = (angle + 360) % 360
        start = (start + 360) % 360
        end = (end + 360) % 360
        if start <= end:
            return start <= angle <= end
        else:
            return angle >= start or angle <= end
    
    def check_plane_alignment(self, normal):
        result = {}

        zenith_angle = np.degrees(np.arccos(np.clip(normal[2], -1, 1)))

        for plane_id in [0, self.bend_id]:
            bins = self.plane_dominant_bins[plane_id]
            az_bins = bins.get("azimuth", [])
            zen_bins = bins.get("zenith", [])

            # Always check zenith
            zen_match = self.is_normal_in_any_bin(normal, zen_bins, "zenith") if zen_bins else True

            # Only check azimuth if zenith is not ~0 or ~180
            if 2 < zenith_angle < 178:
                az_match = self.is_normal_in_any_bin(normal, az_bins, "azimuth") if az_bins else True
            else:
                az_match = True  # skip azimuth check for vertical normals

            result[plane_id] = az_match and zen_match

        return result
    
    def find_hemisphere_normal(self, neighborhood_points):
        mean = np.mean(neighborhood_points, axis=0)
        centered = neighborhood_points - mean
        cov = np.cov(centered.T)
        eig_vals, eig_vecs = np.linalg.eigh(cov)
        eig_vecs = eig_vecs[:, np.argsort(eig_vals)]  # ascending order

        pca_normal = eig_vecs[:, 0]  # Smallest eigenvalue → normal direction
        return pca_normal
    
    def align_normals(self, reference_normal, neighbor_directions):
        aligned_normals = np.array(neighbor_directions)
        
        # Check dot product: If negative, flip the normal
        for i in range(len(aligned_normals)):
            if np.dot(reference_normal, aligned_normals[i]) < 0:
                aligned_normals[i] = -aligned_normals[i]

        return aligned_normals

    def compute_piecewise_arc_length(self, points):
        return np.sum(np.linalg.norm(np.diff(points, axis=0), axis=1))

    def hug_surface(self, last_bend_point, direction_vector): 
        # 1. Find k-NN from the bend cluster (or pcd, if more stable)
        neighbor_indices = self.bend_cluster_kdtree.query(last_bend_point, k=15)[1]
        neighbor_points = self.bend_cluster_points[neighbor_indices]

        # 2. Fit a PCA plane
        mean = np.mean(neighbor_points, axis=0)
        centered = neighbor_points - mean
        cov = np.cov(centered.T)
        eig_vals, eig_vecs = np.linalg.eigh(cov)
        eig_vecs = eig_vecs[:, np.argsort(eig_vals)]
        pca_normal = eig_vecs[:, 0]  # Normal to the surface

        # Flip the normal if it's facing the wrong way
        vec_to_point = last_bend_point - mean
        if np.dot(vec_to_point, pca_normal) < 0:
            pca_normal = -pca_normal

        # 3. Project the raw point onto the PCA plane
        distance = np.dot(vec_to_point, pca_normal)
        projected_point = last_bend_point - distance * pca_normal

        # 4. Constrain the correction to be perpendicular to the derivative direction
        correction_vec = projected_point - last_bend_point
        perp_component = correction_vec - np.dot(correction_vec, direction_vector) * direction_vector

        # 5. Apply only the perpendicular correction
        final_point = last_bend_point + perp_component

        return final_point
    
    def find_bend_end(self, bend_point, derivate_direction):
        initial_direction = derivate_direction.copy()
        path_points = [bend_point.copy()]
        i=0
        while True:
            neighbor_all_indices = self.pcd_kdtree.query_ball_point(bend_point, self.radius)
            if len(neighbor_all_indices) > 5:
                # Separate neighbors by hemisphere
                neighbor_points = self.points[neighbor_all_indices]
                # FULL HEMISPHERE
                # vectors = neighbor_points - bend_point
                # dot_products = vectors @ derivate_direction
                # forward_mask = dot_products > 0
                # forward_indices = np.array(neighbor_all_indices)[forward_mask]
                # #print(f"Forward indices: {forward_indices}")
                # forward_points = self.points[forward_indices]
                # pca_normal = self.find_hemisphere_normal(forward_points)

                # SLAB FORWARD STRUCTURE
                vectors = neighbor_points - bend_point
                dot_products = vectors @ derivate_direction
                epsilon = 0.1
                mask_1 = np.abs(dot_products) < epsilon
                mask_2 = dot_products > 0
                final_mask = mask_1 & mask_2
                #final_mask = mask_1
                forward_indices = np.array(neighbor_all_indices)[final_mask]
                forward_points = self.points[forward_indices]
                pca_normal = self.find_hemisphere_normal(forward_points)

                bend_global_idx = self.pcd_kdtree.query(bend_point)[1]
                original_normal = self.all_normals[bend_global_idx]

                if np.dot(pca_normal, original_normal) < 0:
                    pca_normal = -pca_normal

                alignment = self.check_plane_alignment(pca_normal)
                #print(f"Alignment: Plane 0 = {alignment[0]}, Plane {self.bend_id} = {alignment[self.bend_id]}", flush=True)
                if alignment[0] or alignment[self.bend_id]:
                    #print("Found alignment in direction", derivate_direction)
                    total_arc_length = self.compute_piecewise_arc_length(np.array(path_points))
                    return projected_bend_point, total_arc_length
                neighbor_bend_indices = self.bend_cluster_kdtree.query_ball_point(bend_point, self.radius)
                if len(neighbor_bend_indices) > 5:
                    neighbor_derivatives = self.bend_cluster_derivatives[neighbor_bend_indices]
                    aligned_neighbor_derivatives = self.align_normals(initial_direction, neighbor_derivatives)
                    derivate_direction = np.mean(aligned_neighbor_derivatives, axis=0)
                    derivate_direction /= np.linalg.norm(derivate_direction)
                bend_point = bend_point + self.derivative_step_size*derivate_direction
                projected_bend_point = self.hug_surface(bend_point, derivate_direction)
                path_points.append(projected_bend_point.copy())
                bend_point = projected_bend_point.copy()
                #print(f'i = {i}')
                i += 1
            else:
                total_arc_length = self.compute_piecewise_arc_length(np.array(path_points))
                return projected_bend_point, total_arc_length
            
    def calculate_arc_length(self):
        # At the top of the function
        results_table = []
        distances = []
        direction_vector = self.intersection_line[0] / np.linalg.norm(self.intersection_line[0])
        to_end = self.end_point - self.start_point
        if np.dot(direction_vector, to_end) < 0:
            direction_vector *= -1  # Flip if it's pointing toward the wrong side
        
        offset = self.radius
        sampled_point = self.start_point + 2 * offset * direction_vector
        end_points_forward = []
        end_points_backward = []
        while np.linalg.norm(sampled_point - self.start_point) < np.linalg.norm(self.end_point - self.start_point) - 2* offset:
            # Sample next point
            sampled_point += self.intersection_step_size * direction_vector
            # Get point closest to the sampled point
            idx = self.bend_cluster_kdtree.query(sampled_point)[1]
            base = self.bend_cluster_points[idx]
            # Get the average derivative of the bend in the neighborhood
            neighbor_bend_indices = self.bend_cluster_kdtree.query_ball_point(base, self.radius)
            neighbor_derivatives = self.bend_cluster_derivatives[neighbor_bend_indices]
            aligned_neighbor_derivatives = self.align_normals(neighbor_derivatives[0], neighbor_derivatives)
            derivate_reference = np.mean(aligned_neighbor_derivatives, axis=0)
            derivate_reference /= np.linalg.norm(derivate_reference)

            # Calculate bend end point in forward direction
            end_point_forward, first_half_distance = self.find_bend_end(base, derivate_reference)
            
            # Calculate bend end point in backward direction
            end_point_backward, second_half_distance = self.find_bend_end(base, -derivate_reference)

            end_points_forward.append(end_point_forward)
            end_points_backward.append(end_point_backward)

            total_distance = first_half_distance + second_half_distance
            distances.append(total_distance)
            results_table.append({
                "Sample Index": len(results_table),
                "Base XYZ": base,
                "Arc Length Forward": first_half_distance,
                "Arc Length Backward": second_half_distance,
                "Total Arc Length": total_distance
            })

        # green = [0.0, 1.0, 0.0]
        # #Visualize the points
        # for point in end_points_forward:
        #     idx = self.pcd_kdtree.query(point)[1]
        #     np.asarray(self.pcd.colors)[idx] = green

        # for point in end_points_backward:
        #     idx = self.pcd_kdtree.query(point)[1]
        #     np.asarray(self.pcd.colors)[idx] = green

        # self.pcd.colors = o3d.utility.Vector3dVector(np.asarray(self.pcd.colors))
        # self.vis.update_geometry(self.pcd)

        # Convert to numpy arrays
        # new_points = np.array(end_points_forward + end_points_backward)

        # # Assign color (green for all)
        # green = np.array([[0.0, 0.0, 1.0]] * len(new_points))

        # # Extend point cloud
        # existing_points = np.asarray(self.pcd.points)
        # existing_colors = np.asarray(self.pcd.colors)

        # all_points = np.vstack([existing_points, new_points])
        # all_colors = np.vstack([existing_colors, green])

        # print(f"New points shape: {new_points.shape}")
        # print(f"First few new points:\n{new_points[:5]}")
        # print(f"Green array shape: {green.shape}")
        # print(f"Original points: {existing_points.shape}")
        # print(f"New combined: {all_points.shape}")

        # self.pcd.points = o3d.utility.Vector3dVector(all_points)
        # self.pcd.colors = o3d.utility.Vector3dVector(all_colors)
        # self.vis.update_geometry(self.pcd)
        return pd.DataFrame(results_table)

    # def run(self):
    #     self.vis.run()
    #     self.vis.destroy_window()

In [5]:
results = []

for bend_id, bend_info in bend_data.items():
    print(f"\nProcessing bend ID: {bend_id}")
    calculator = ArcLengthCalculator(
        pcd=pcd,
        bend_id=bend_id,
        intersection_line=bend_info["intersection_line"],
        bend_edges=bend_info["bend_edges"],
        bend_cluster_points=bend_info["cluster_points"],
        cluster_derivatives=bend_info["cluster_derivatives"],
        all_normals=all_normals,
        segment_indices=segment_indices
        
    )
    df = calculator.calculate_arc_length()
    mean_arc = df["Total Arc Length"].mean()
    std_arc = df["Total Arc Length"].std()
    print(f"→ Bend {bend_id}: Mean = {mean_arc:.4f} mm | Std = {std_arc:.4f} mm")
    results.append((bend_id, mean_arc, std_arc))

# Print all results together
print("\n=== Arc Length Summary ===")
for bend_id, mean_arc, std_arc in results:
    print(f"Bend {bend_id}: Mean = {mean_arc:.4f} mm | Std = {std_arc:.4f} mm")


Processing bend ID: 1
Loaded 3060573 normals for 3060573 points in PCD. Proceeding.
→ Bend 1: Mean = 1.6102 mm | Std = 0.0000 mm

Processing bend ID: 2
Loaded 3060573 normals for 3060573 points in PCD. Proceeding.
→ Bend 2: Mean = 1.6102 mm | Std = 0.0000 mm

Processing bend ID: 3
Loaded 3060573 normals for 3060573 points in PCD. Proceeding.
→ Bend 3: Mean = 1.5321 mm | Std = 0.0054 mm

Processing bend ID: 4
Loaded 3060573 normals for 3060573 points in PCD. Proceeding.
→ Bend 4: Mean = 1.5311 mm | Std = 0.0052 mm

Processing bend ID: 5
Loaded 3060573 normals for 3060573 points in PCD. Proceeding.
→ Bend 5: Mean = 1.5314 mm | Std = 0.0053 mm

Processing bend ID: 6
Loaded 3060573 normals for 3060573 points in PCD. Proceeding.
→ Bend 6: Mean = 1.5320 mm | Std = 0.0054 mm

=== Arc Length Summary ===
Bend 1: Mean = 1.6102 mm | Std = 0.0000 mm
Bend 2: Mean = 1.6102 mm | Std = 0.0000 mm
Bend 3: Mean = 1.5321 mm | Std = 0.0054 mm
Bend 4: Mean = 1.5311 mm | Std = 0.0052 mm
Bend 5: Mean = 1.531

In [6]:
pd.set_option('display.max_rows', None)    # Show all rows
pd.set_option('display.max_columns', None) # Show all columns
pd.set_option('display.width', None)       # Don’t wrap lines
pd.set_option('display.max_colwidth', None) # Show full column content

display(df)

,Sample Index,Base XYZ,Arc Length Forward,Arc Length Backward,Total Arc Length
0,0,"[120.56698165647582, 168.09338521400778, 1.1578292846679688]",0.817357,0.709003,1.526360
1,1,"[120.06670372429127, 168.26014452473595, 1.1244964599609375]",0.707547,0.829153,1.536700
2,2,"[119.7331851028349, 168.5936631461923, 1.1629829406738281]",0.837321,0.689080,1.526401
3,3,"[119.23290717065035, 168.7604224569205, 1.1296520233154297]",0.727474,0.808915,1.536389
4,4,"[118.89938854919399, 169.09394107837687, 1.1681442260742188]",0.857776,0.669289,1.527064
5,5,"[118.39911061700944, 169.26070038910504, 1.1348075866699219]",0.747314,0.788752,1.536066
6,6,"[117.8988326848249, 169.5942190105614, 1.1470108032226562]",0.787387,0.748753,1.536140
7,7,"[117.56531406336853, 169.7609783212896, 1.139963150024414]",0.757129,0.768679,1.525808
8,8,"[117.06503613118399, 170.09449694274596, 1.1521644592285156]",0.797253,0.728883,1.526135
9,9,"[116.73151750972762, 170.26125625347413, 1.145120620727539]",0.777238,0.758843,1.536082
